In [3]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML

from api.yelp.yelp import YelpAPI
from algo import *
from utils import *
from constraints import *
from plan import *
from shortcuts import *
import datetime
import time
import pytz
import dateutil.parser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
CAT_FILE = 'api/yelp/yelpcategories.json'
with open('api/yelp/yelpkey.txt', 'r') as f:
    API_KEY = f.read().strip()
api = YelpAPI(API_KEY, CAT_FILE)

In [5]:
addr = "97 5th Avenue New York, NY"
entities = fetch_businesses_by_categories(api=api, 
                                            location=addr, 
                                            categories=["chinese", "speakeasies", "breakfast_brunch"],
                                            num_businesses_per_category=40,
                                            as_entities=True,
                                            request_separately=True)

20 businesses left. Found 20  just now.
20 businesses left. Found 20  just now.
{'b5': {'name': 'Excellent Dumpling House', 'id': '7ZQSMqWzvXMSetXnii2fBg', 'hours': [{'open': [{'end': '2230', 'day': 0, 'start': '1100', 'is_overnight': False}, {'end': '2230', 'day': 1, 'start': '1100', 'is_overnight': False}, {'end': '2230', 'day': 2, 'start': '1100', 'is_overnight': False}, {'end': '2230', 'day': 3, 'start': '1100', 'is_overnight': False}, {'end': '2300', 'day': 4, 'start': '1100', 'is_overnight': False}, {'end': '2300', 'day': 5, 'start': '1100', 'is_overnight': False}, {'end': '2230', 'day': 6, 'start': '1100', 'is_overnight': False}], 'is_open_now': True}]}, 'b4': {'name': 'Tim Ho Wan', 'id': 'SyvC0s3y0ZrtnLbwnvw7tw', 'hours': [{'open': [{'end': '2200', 'day': 0, 'start': '1000', 'is_overnight': False}, {'end': '2200', 'day': 1, 'start': '1000', 'is_overnight': False}, {'end': '2200', 'day': 2, 'start': '1000', 'is_overnight': False}, {'end': '2200', 'day': 3, 'start': '1000', 'is_o

In [8]:
seq = None
G = build_entity_graph(entities, seq, min_rating=4.0, max_distance=2000, min_dollar_signs=1, max_dollar_signs=2)
u = UberConstraints()
b = BooleanConstraints()
se = StateExtractors()
sc = StateConstraints()
ee = EdgeExtractors()
neighbor_constraint = b.bool_and(
    [
        b.bool_less_than(se.get_attr("num_prev_states"), se.const(1)),
        u.follows_cat_sequence([set([x]) for x in ["breakfast_brunch"]]),
        b.bool_less_than(ee.get_distance(use_miles=True), se.const(10.75)),
        sc.no_repeat_visits()
    ]
)

def time_spent_fn(entity):
    if "movietheaters" in entity.entity.categories:
        return [datetime.timedelta(hours=3)]
    return [datetime.timedelta(hours=1.5)]

done_constraint = b.bool_equals(se.get_attr("num_prev_states"), se.const(0))
init_constraint = sc.curr_state_category_in(set(["breakfast_brunch"]))

date_start = datetime.datetime(2019, 3, 2, 12, 30)
initial_states = build_initial_states(G, [date_start], init_constraint)

neighbor_fn = build_neighbor_state_fn(neighbor_constraint, time_spent_fn=time_spent_fn)

success_fn = build_success_state_fn(done_constraint)


In [9]:
results = list(generate_plans(G, initial_states, neighbor_fn, success_fn, None))
results = sort_plans_by(results, lambda x: plan_total_distance(G, x))

In [10]:
def url_ver(entity):
    img = '<img src="{}" style="width:80px; display:inline"></img>'.format(entity.image_url)
    return img +  '<a href="{}">{}</a>'.format(entity.url, entity.name)

def print_results(results):
    # t_format = "%m/%d/%y %B %Y %H:%M"
    t_format = "%I:%M %p"
    for state in results:
        all_prev = state.prev_state_list()
        maps_link = build_maps_link([s.node.entity.name + " " + s.node.get_address() for s in all_prev] + [state.node.entity.name + " " + state.node.get_address()], "walking")
        maps_link = '<a href="{}">(Maps Link)</a>'.format(maps_link)
        out = []
        for s in (all_prev + [state]):
            out.append("{} ({} - {})".format(url_ver(s.node.entity), s.start_dt.strftime(t_format), s.end_dt.strftime(t_format)))
        res = " => ".join(out)
        display(HTML(res + " " + maps_link))
        print()
print_results(results)

In [76]:
meters_to_miles(G.get_edge_properties(results[-1].prev_state_list()[0].node, results[-1].prev_state_list()[1].node)['dist'])

IndexError: list index out of range

In [52]:
results[0].node.get_address()

'204 W 55th St New York, NY 10019'

98